#라이브러리

In [ ]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
from PIL import Image

#BackBone
Backbone(Feature Extractor)
Detection, Segmentation 등 고도화된 이미지 처리 인공지능 모델에서, 이미지 특징 추출을 위해 사용되는 Image Classification 모델

Pretrained: ImageNet 등 데이터셋에 대해 사전 학습된 모델의 가중치(weight)
Backbone을 사용해야 하는 이유
Object detection의 Faster R-CNN, Libra R-CNN의 경우, backbone 모델에 따라 판독 성능이 차이나는 것을 확인할 수 있음

ResNet-50-FPN, ResNet-101-FPN, ResNeXt-101-FPN 순으로 정확도 향상


ImageNet 기반에서 1,2% 성능 차이가 나는 경우 실제 모델 적용 상황(targettask)에서도 2~3% 이상 성능 차이 발생

그림의 왼쪽, 오른쪽 column은 ImageNet Classification에서 1,2% 성능 차이가 나는 모델로, COCO2017에서 backbone으로 사용되었을 때 2~3% 성능 차이를 보였으나, 실제 detection 결과는 매우 차이남

더 좋은 Backbone을 사용한다면 더 높은 detection 수준을 가질 것으로 기대

In [ ]:
# 분류를 위한 사전 훈련된 모델 로드하고, 특징만 반환합니다.
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN은 백본의 출력 채널 수를 알아야 합니다. mobilenet_v2의 경우, 1280입니다.
backbone.out_channels = 1280


#RPN(Region Proposal Network) 정의


In [ ]:
# RPN이 공간 위치마다 5개의 크기와 3개의 가로세로 비율을 가진 5 x 3 앵커를 생성하도록 합니다.
# 각 피처 맵이 다른 크기와 가로세로 비율을 가질 수 있기 때문에 Tuple[Tuple[int]]을 사용합니다.
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# 관심 영역(ROI) 크롭핑을 수행할 피처 맵과, 크롭 후의 사이즈를 정의합니다.
# 백본이 Tensor를 반환한다면 featmap_names는 [0]이 될 것입니다.
# 일반적으로, 백본은 OrderedDict[Tensor]를 반환해야 하며,
# featmap_names에서 사용할 피처 맵을 선택할 수 있습니다.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                 output_size=7,
                                                 sampling_ratio=2)

In [ ]:
# FasterRCNN 모델 안에 조각들을 조합합니다.
model = FasterRCNN(backbone,
                   num_classes=2,  # 배경을 포함한 클래스 수
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

In [ ]:
# 시연을 위해 이미지 하나를 로드합니다.
img = Image.open("path_to_your_image.jpg")
# 이미지를 텐서로 변환합니다.
img_tensor = F.to_tensor(img)

# 모델을 평가 모드로 설정합니다.
model.eval()
with torch.no_grad():
    prediction = model([img_tensor])